In [3]:
import os
ultrasonic_csv_folder = "//mnt/sdb/ywatanabe/csv_files"
print(os.listdir(ultrasonic_csv_folder))

['2023_2dim_hilbert_tdx2', '2023_2dim_rawcsv_tdx1_trans', 'camera_data', '2023_2dim_hilbert_tdx1', '2023_2dim_hilbert_tdx1_trans', '2023_2dim_rawcsv_tdx1']


In [10]:
import polars as pl
import pandas as pd
rawcsv_folder = "//mnt/sdb/ywatanabe/csv_files/2023_2dim_rawcsv_tdx1"
sample = pd.read_csv(rawcsv_folder+"/s1_g0_l1_t1_tdx1.csv")
print(sample.shape)

(14997, 4012)


In [20]:
import pandas as pd
import numpy as np
import librosa

# row_index 番目の行の波形データを取得 (0番目の列は時刻なので除外)
row_index = 0
waveform = sample.iloc[row_index, 1:].values.astype(float)  # 1列目以降をfloat配列化

# サンプリングレート(仮に 44100 とする)
sr = 4410

# librosa で MFCC を抽出（n_mfcc は取りたい MFCC 次元数に応じて設定）
mfcc = librosa.feature.mfcc(y=waveform, sr=sr, n_mfcc=1)
print(waveform.shape)
print(mfcc.shape)
print(mfcc)


(4011,)
(1, 8)
[[623.57651659 599.31767937 451.95158983 476.78374736 518.39215141
  489.62685066 441.45822236 452.80823909]]


In [21]:
import pandas as pd
import numpy as np
import librosa

# 例: DataFrame "sample" の各行に波形データが入っているとする
#  - 0番目の列: 時刻 or IDなど
#  - 1番目以降: 波形の振幅データ
# たとえば sample.shape == (行数, N+1) など

# サンプリングレート (例として 4410)
sr = 4410  
# 取りたい MFCC 次元数
n_mfcc = 13

mfcc_list = []

for i in range(len(sample)):
    # 行 i の波形データ（1列目以降）を float 配列に変換
    waveform = sample.iloc[i, 1:].values.astype(float)
    
    # MFCC を抽出: shape = (n_mfcc, T)
    mfcc = librosa.feature.mfcc(y=waveform, sr=sr, n_mfcc=n_mfcc)
    
    # そのまま 3次元 (サンプル数, n_mfcc, T) で扱いたい場合
    # mfcc_list.append(mfcc[np.newaxis, :, :])

    # あるいは 2次元 (サンプル数, n_mfcc*T) にフラット化したい場合
    mfcc_flat = mfcc.flatten()
    mfcc_list.append(mfcc_flat)

# =========================
# (A) 3次元にまとめる例
# =========================
# mfcc_list には (1, n_mfcc, T) がサンプル数個入っている想定の場合
# mfcc_all = np.concatenate(mfcc_list, axis=0)
# → shape = (サンプル数, n_mfcc, T)

# =========================
# (B) 2次元にまとめる例
# =========================
# mfcc_list にはフラット化済みベクトルがサンプル数個入っている
mfcc_all = np.array(mfcc_list)
# shape = (サンプル数, n_mfcc*T)

print("MFCC 行列の形状:", mfcc_all.shape)


MFCC 行列の形状: (14997, 104)


In [22]:
import os
import pandas as pd
import numpy as np
import librosa

def convert_csv_to_mfcc_npy(input_dir, output_dir, sr=44100, n_mfcc=13):
    """
    input_dir  : CSVファイルが存在するディレクトリ
    output_dir : 変換した .npy を保存するディレクトリ
    sr         : サンプリングレート
    n_mfcc     : 取得するMFCCの次元数
    """

    # 出力フォルダがなければ作成
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # ディレクトリ内のすべての .csv ファイルを取得
    csv_files = [f for f in os.listdir(input_dir) if f.endswith(".csv")]

    for csv_file in csv_files:
        input_path = os.path.join(input_dir, csv_file)
        output_path = os.path.join(output_dir, csv_file + ".npy")
        
        # CSV 読み込み (例: 最初の列に時刻やID等があり、それ以外が波形データだと想定)
        df = pd.read_csv(input_path)
        
        # 波形部分を float に変換
        #  - ここでは、全行・「最初の列を除いたすべての列」を結合して 1D 波形とみなす例
        #    CSV の構造に合わせて適宜修正してください。
        waveform = df.iloc[:, 1:].values.flatten().astype(float)
        
        # MFCC を取得  shape = (n_mfcc, T)
        mfcc = librosa.feature.mfcc(y=waveform, sr=sr, n_mfcc=n_mfcc)
        
        # .npy 形式で出力保存
        np.save(output_path, mfcc)
        print(f"Saved MFCC to: {output_path} (shape: {mfcc.shape})")


if __name__ == "__main__":
    # 入力CSVがあるディレクトリ
    input_dir = "/mnt/sdb/ywatanabe/csv_files/2023_2dim_rawcsv_tdx1"

    # 出力先ディレクトリ (なければ自動生成)
    output_dir = "/mnt/sdb/matsubara/csv_files/mfcc_npy_files"

    # サンプリングレートや MFCC 次元数は必要に応じて変更
    sr = 4410
    n_mfcc = 13

    convert_csv_to_mfcc_npy(
        input_dir=input_dir,
        output_dir=output_dir,
        sr=sr,
        n_mfcc=n_mfcc
    )


Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s2_g0_l3_t4_tdx1.csv.npy (shape: (13, 117487))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s3_g0_l1_t5_tdx1.csv.npy (shape: (13, 117487))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s2_g0_l1_t5_tdx1.csv.npy (shape: (13, 117487))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l2_t1_tdx1.csv.npy (shape: (13, 117487))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l1_t5_tdx1.csv.npy (shape: (13, 117487))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l3_t2_tdx1.csv.npy (shape: (13, 117487))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l3_t4_tdx1.csv.npy (shape: (13, 117487))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s2_g0_l1_t4_tdx1.csv.npy (shape: (13, 117487))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l1_t3_tdx1.csv.npy (shape: (13, 117487))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mf

In [33]:
import os
import pandas as pd
import numpy as np
import librosa

def convert_csv_to_mfcc_npy(input_dir, output_dir, sr=44100, n_mfcc=104):
    """
    input_dir  : CSVファイルが存在するディレクトリ
    output_dir : 変換した .npy を保存するディレクトリ
    sr         : サンプリングレート
    n_mfcc     : 取得するMFCCの次元数(ここでは104)
    """

    # 出力フォルダがなければ作成
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # ディレクトリ内のすべての .csv ファイルを取得
    csv_files = [f for f in os.listdir(input_dir) if f.endswith(".csv")]

    for csv_file in csv_files:
        input_path = os.path.join(input_dir, csv_file)
        
        # 拡張子 .csv を取り除いたファイル名を取得
        base_name = os.path.splitext(csv_file)[0]
        # 出力ファイルパス (".csv"を含まない形にして ".npy" を付与)
        output_path = os.path.join(output_dir, base_name + ".npy")
        
        # CSV 読み込み
        # 例: 最初の列は時刻やIDなどと想定し、それ以外が波形データ
        df = pd.read_csv(input_path)
        
        # 波形部分を float に変換
        # CSVの構造に合わせて適宜修正してください
        waveform = df.iloc[:, 1:].values.flatten().astype(float)
        
        # MFCC を取得: shape = (n_mfcc, T)
        mfcc = librosa.feature.mfcc(y=waveform, sr=sr, n_mfcc=n_mfcc)
        
        # 形状を (T, n_mfcc) に転置  -> 例: (15000程度, 104)
        mfcc = mfcc.T
        
        # .npy 形式で出力保存
        np.save(output_path, mfcc)
        print(f"Saved MFCC to: {output_path} (shape: {mfcc.shape})")


if __name__ == "__main__":
    # 入力CSVがあるディレクトリ
    input_dir = "/mnt/sdb/ywatanabe/csv_files/2023_2dim_rawcsv_tdx1"

    # 出力先ディレクトリ
    output_dir = "/mnt/sdb/matsubara/csv_files/mfcc_npy_files"

    # サンプリングレートやMFCC次元を実際の仕様にあわせる
    sr = 4410       # 例
    n_mfcc = 104    # 例: 104次元

    convert_csv_to_mfcc_npy(
        input_dir=input_dir,
        output_dir=output_dir,
        sr=sr,
        n_mfcc=n_mfcc
    )


Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s2_g0_l3_t4_tdx1.npy (shape: (117487, 104))


KeyboardInterrupt: 

In [1]:
import numpy as np 
sample = np.load("//mnt/sdb/matsubara/csv_files/mfcc_npy_files/s2_g0_l1_t5_tdx1.npy")
print(sample.shape)

(300765, 104)


In [36]:
import os
import pandas as pd
import numpy as np
import torch
import torchaudio

def convert_csv_to_mfcc_npy_torchaudio(input_dir, output_dir, sr=44100, n_mfcc=104):
    """
    input_dir  : CSVファイルが存在するディレクトリ
    output_dir : 変換した .npy を保存するディレクトリ
    sr         : サンプリングレート
    n_mfcc     : 取得するMFCCの次元数 (例: 104)
    """

    # GPU が利用可能ならGPUを使う、なければCPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    # 出力フォルダ作成
    os.makedirs(output_dir, exist_ok=True)
    
    # ディレクトリ内のすべての .csv ファイルを取得
    csv_files = [f for f in os.listdir(input_dir) if f.endswith(".csv")]
    csv_files.sort()

    # torchaudio の MFCC 変換モジュールを作成 (GPU 上に配置)
    mfcc_transform = torchaudio.transforms.MFCC(
        sample_rate=sr,
        n_mfcc=n_mfcc
    ).to(device)

    for csv_file in csv_files:
        input_path = os.path.join(input_dir, csv_file)
        
        # 拡張子 .csv を取り除いたファイル名を取得
        base_name = os.path.splitext(csv_file)[0]
        # 出力ファイルパス (".csv" を含まない形にして ".npy" を付与)
        output_path = os.path.join(output_dir, base_name + ".npy")
        
        # CSV を読み込み
        df = pd.read_csv(input_path)
        
        # 最初の列が時刻やID等、2列目以降が波形データを想定
        waveform_np = df.iloc[:, 1:].values.flatten().astype(np.float32)
        
        # NumPy配列 → PyTorch Tensor (shape: [T]) に変換 → [1, T] に拡張してモノラル扱い
        waveform_tensor = torch.from_numpy(waveform_np).unsqueeze(0).to(device)  # [1, T]

        # MFCC 変換 (shape: [batch=1, n_mfcc, time_frames])
        mfcc_tensor = mfcc_transform(waveform_tensor)

        # バッチ次元を削除 → (n_mfcc, time_frames) → 転置: (time_frames, n_mfcc)
        mfcc_tensor = mfcc_tensor.squeeze(0).transpose(0, 1)

        # CPU に戻して .npy 保存
        mfcc = mfcc_tensor.cpu().numpy()
        np.save(output_path, mfcc)

        print(f"Saved MFCC to: {output_path} (shape: {mfcc.shape})")


if __name__ == "__main__":
    # 入力CSVがあるディレクトリ
    input_dir = "/mnt/sdb/ywatanabe/csv_files/2023_2dim_rawcsv_tdx1"
    # 出力先ディレクトリ
    output_dir = "/mnt/sdb/matsubara/csv_files/mfcc_npy_files"

    # サンプリングレートや MFCC の次元を実際の仕様に合わせて設定
    sr = 4410       
    n_mfcc = 12  

    convert_csv_to_mfcc_npy_torchaudio(
        input_dir=input_dir,
        output_dir=output_dir,
        sr=sr,
        n_mfcc=n_mfcc
    )


Using device: cuda
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l1_t1_tdx1.npy (shape: (300765, 104))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l1_t2_tdx1.npy (shape: (300765, 104))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l1_t3_tdx1.npy (shape: (300765, 104))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l1_t4_tdx1.npy (shape: (300765, 104))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l1_t5_tdx1.npy (shape: (300765, 104))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l2_t1_tdx1.npy (shape: (300765, 104))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l2_t2_tdx1.npy (shape: (300765, 104))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l2_t3_tdx1.npy (shape: (300765, 104))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_files/s1_g0_l2_t4_tdx1.npy (shape: (300765, 104))
Saved MFCC to: /mnt/sdb/matsubara/csv_files/mfcc_npy_f

In [5]:

import os
import pandas as pd
import numpy as np

def excel_to_npy(input_excel, output_csv):
    """
    ExcelファイルのA列にID、B列に目標変数、C列に実験条件IDが記載されていると想定し、
    それら3つの列をNumPy配列にまとめて .npy 形式で保存する。

    Parameters
    ----------
    input_excel : str
        入力するExcelファイル(.xlsxなど)へのパス
    output_csv : str
        出力する .csv ファイルのパス
    """
    # Excelを読み込み（デフォルトで最初のシートを読む）
    # 読み込んだDataFrameの0列目(A列)がID, 1列目(B列)が目標変数, 2列目(C列)が実験条件IDと想定
    df = pd.read_excel(input_excel, header=None)

    # A列, B列, C列を取り出し、(行数, 3列) の2次元配列を作成
    # 例: df.iloc[:, 0] が A列, df.iloc[:, 1] が B列, df.iloc[:, 2] が C列
    data_array = df.iloc[:, [0, 5, 2]]

    # .npy 形式で保存
    #np.save(output_csv, data_array)
    data_array.to_csv(output_csv,index=False)
    print(f"Saved data to: {output_csv}")
    print("Data shape:", data_array.shape)


if __name__ == "__main__":
    # 入力Excelファイルと出力ファイルを指定
    # 例: A列=ID, B列=目標変数, C列=実験条件ID
    input_excel = "/home/user01/download/固液二相流(送付用).xlsx"
    output_csv = "//mnt/sdb/matsubara/target/t_data.csv"

    excel_to_npy(input_excel, output_csv)


Saved data to: //mnt/sdb/matsubara/target/t_data.csv
Data shape: (71, 3)


In [19]:
import pandas as pd
sample =pd.read_csv("//mnt/sdb/matsubara/target/t_data.csv")
"another = df.dropna(subset=[2])
print(sample.shape)
"print(another.head())

SyntaxError: unterminated string literal (detected at line 3) (443796529.py, line 3)

In [18]:
import os
import pandas as pd
import numpy as np
import torch
import torchaudio

def compute_scalar_from_waveform(waveform, sr=44100, n_mfcc=20):
    """
    例: 波形に対して torchaudio で MFCC を計算し、
        その全要素の平均をスカラー値として返す。
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    mfcc_transform = torchaudio.transforms.MFCC(
        sample_rate=sr,
        n_mfcc=n_mfcc
    ).to(device)

    waveform_tensor = torch.from_numpy(waveform).unsqueeze(0).float().to(device)
    mfcc_tensor = mfcc_transform(waveform_tensor)
    mfcc_tensor = mfcc_tensor.squeeze(0)  # shape: (n_mfcc, time_frames)

    scalar_value = mfcc_tensor.mean().item()
    return scalar_value

def process_all_csv(input_dir, output_csv, sr=44100, n_mfcc=20):
    """
    - 指定ディレクトリ(input_dir)にある .csv ファイルを全て走査
    - 各ファイルについて:
      1) すべての行に対して音響特徴量 (スカラー)を計算
      2) 行ごとの特徴量を平均
      3) (ファイル名, 平均特徴量) として出力用リストに1行追加
    - 最後に (ファイル数行) のDataFrameを CSV出力する。
    """

    csv_files = [f for f in os.listdir(input_dir) if f.endswith(".csv")]
    csv_files.sort()

    # 結果を格納するリスト (ファイル単位で1行ずつ)
    output_data = []

    for csv_file in csv_files:
        file_path = os.path.join(input_dir, csv_file)
        print(f"Processing: {file_path}")

        # CSV 読み込み: A列(時刻), B列以降(波形)
        df = pd.read_csv(file_path, header=None)

        # ファイル内の行ごとの特徴量を計算して蓄積
        scalar_values = []
        for i in range(len(df)):
            # B列以降の波形を取り出し
            row_waveform = df.iloc[i, 1:].values.flatten().astype(np.float32)

            # 行ごとの特徴量を計算
            val = compute_scalar_from_waveform(row_waveform, sr=sr, n_mfcc=n_mfcc)
            scalar_values.append(val)

        # ファイル内の全行(=全測定)の特徴量を平均
        if len(scalar_values) > 0:
            mean_val = float(np.mean(scalar_values))
        else:
            mean_val = float('nan')

        # 出力フォーマット: [ケース名(CSVファイル), 平均特徴量]
        output_data.append([csv_file, mean_val])

    # 最終的に DataFrame 化 → CSV 出力
    out_df = pd.DataFrame(output_data, columns=["SourceCSV", "MeanFeatureValue"])
    out_df.to_csv(output_csv, index=False, encoding="utf-8")

    print(f"Done. Saved to {output_csv}")

if __name__ == "__main__":
    input_dir = "//mnt/sdb/ywatanabe/csv_files/2023_2dim_rawcsv_tdx1"
    output_csv = "//mnt/sdb/matsubara/input/all_features.csv"

    sr = 44100  # サンプリングレート (実際に合わせて調整)
    n_mfcc = 20 # MFCC次元 (必要に応じて変更)

    process_all_csv(input_dir, output_csv, sr=sr, n_mfcc=n_mfcc)


Processing: //mnt/sdb/ywatanabe/csv_files/2023_2dim_rawcsv_tdx1/s1_g0_l1_t1_tdx1.csv


/home/user01/Document/Matsubara/myenv/lib/python3.10/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


Processing: //mnt/sdb/ywatanabe/csv_files/2023_2dim_rawcsv_tdx1/s1_g0_l1_t2_tdx1.csv


/home/user01/Document/Matsubara/myenv/lib/python3.10/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


Processing: //mnt/sdb/ywatanabe/csv_files/2023_2dim_rawcsv_tdx1/s1_g0_l1_t3_tdx1.csv


...

Processing: //mnt/sdb/ywatanabe/csv_files/2023_2dim_rawcsv_tdx1/s3_g0_l3_t3_tdx1.csv


/home/user01/Document/Matsubara/myenv/lib/python3.10/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


Processing: //mnt/sdb/ywatanabe/csv_files/2023_2dim_rawcsv_tdx1/s3_g0_l3_t4_tdx1.csv


/home/user01/Document/Matsubara/myenv/lib/python3.10/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


Done. Saved to //mnt/sdb/matsubara/input/all_features.csv


In [28]:
import pandas as pd
df = pd.read_csv("//mnt/sdb/matsubara/input/all_features.csv")
df.head

<bound method NDFrame.head of                SourceCSV  MeanFeatureValue
0   s1_g0_l1_t1_tdx1.csv          8.415898
1   s1_g0_l1_t2_tdx1.csv          8.421776
2   s1_g0_l1_t3_tdx1.csv          8.640985
3   s1_g0_l1_t4_tdx1.csv          8.608584
4   s1_g0_l1_t5_tdx1.csv          8.573729
5   s1_g0_l2_t1_tdx1.csv          8.385362
6   s1_g0_l2_t2_tdx1.csv          8.402472
7   s1_g0_l2_t3_tdx1.csv          8.442112
8   s1_g0_l2_t4_tdx1.csv          8.593244
9   s1_g0_l2_t5_tdx1.csv          8.603125
10  s1_g0_l3_t2_tdx1.csv          8.319590
11  s1_g0_l3_t3_tdx1.csv          8.329208
12  s1_g0_l3_t4_tdx1.csv          8.746826
13  s1_g0_l3_t5_tdx1.csv          8.635511
14  s1_g0_l3_t6_tdx1.csv          8.622119
15  s2_g0_l1_t1_tdx1.csv          8.506963
16  s2_g0_l1_t2_tdx1.csv          8.552431
17  s2_g0_l1_t4_tdx1.csv          8.438498
18  s2_g0_l1_t5_tdx1.csv          8.508600
19  s2_g0_l1_t6_tdx1.csv          8.494484
20  s2_g0_l2_t1_tdx1.csv          8.415078
21  s2_g0_l2_t2_tdx1.csv

In [31]:
import pandas as pd

def standardize_and_add_third_column(input_csv, output_csv):
    # CSV を読み込み（列名が無い場合は適宜設定）
    df = pd.read_csv(input_csv)
    
    # 2列目 "MeanFeatureValue" の平均・標準偏差を計算
    mean_val = df["MeanFeatureValue"].mean()
    std_val = df["MeanFeatureValue"].std()

    # 新しい列(3列目) として標準化した値を追加
    df["FeatureStd"] = (df["MeanFeatureValue"] - mean_val) / std_val
    
    # 確認
    print(df.head())
    print(df.shape)  # shape は (44, 3) になるはず
    
    # CSV 出力（列名を含めたい場合はデフォルトのままでOK）
    df.to_csv(output_csv, index=False)

if __name__ == "__main__":
    input_csv = "//mnt/sdb/matsubara/input/all_features.csv"
    output_csv = "//mnt/sdb/matsubara/input/all_features_standardized.csv"

    standardize_and_add_third_column(input_csv, output_csv)
    print("標準化が完了しました。")


              SourceCSV  MeanFeatureValue  FeatureStd
0  s1_g0_l1_t1_tdx1.csv          8.415898    0.102884
1  s1_g0_l1_t2_tdx1.csv          8.421776    0.110117
2  s1_g0_l1_t3_tdx1.csv          8.640985    0.379857
3  s1_g0_l1_t4_tdx1.csv          8.608584    0.339987
4  s1_g0_l1_t5_tdx1.csv          8.573729    0.297098
(44, 3)
標準化が完了しました。


In [35]:
import pandas as pd
df = pd.read_csv("//mnt/sdb/matsubara/input/all_features.csv")

sample =pd.read_csv("//mnt/sdb/matsubara/target/t_data.csv")
print(df.shape)
print(sample.shape)
df.head


(44, 2)
(71, 3)


<bound method NDFrame.head of                SourceCSV  MeanFeatureValue
0   s1_g0_l1_t1_tdx1.csv          8.415898
1   s1_g0_l1_t2_tdx1.csv          8.421776
2   s1_g0_l1_t3_tdx1.csv          8.640985
3   s1_g0_l1_t4_tdx1.csv          8.608584
4   s1_g0_l1_t5_tdx1.csv          8.573729
5   s1_g0_l2_t1_tdx1.csv          8.385362
6   s1_g0_l2_t2_tdx1.csv          8.402472
7   s1_g0_l2_t3_tdx1.csv          8.442112
8   s1_g0_l2_t4_tdx1.csv          8.593244
9   s1_g0_l2_t5_tdx1.csv          8.603125
10  s1_g0_l3_t2_tdx1.csv          8.319590
11  s1_g0_l3_t3_tdx1.csv          8.329208
12  s1_g0_l3_t4_tdx1.csv          8.746826
13  s1_g0_l3_t5_tdx1.csv          8.635511
14  s1_g0_l3_t6_tdx1.csv          8.622119
15  s2_g0_l1_t1_tdx1.csv          8.506963
16  s2_g0_l1_t2_tdx1.csv          8.552431
17  s2_g0_l1_t4_tdx1.csv          8.438498
18  s2_g0_l1_t5_tdx1.csv          8.508600
19  s2_g0_l1_t6_tdx1.csv          8.494484
20  s2_g0_l2_t1_tdx1.csv          8.415078
21  s2_g0_l2_t2_tdx1.csv

In [36]:
import os
import re
import pandas as pd

def match_and_order(
    t_data_csv: str,
    all_features_csv: str,
    output_csv: str
):
    """
    t_data_csv:   計測日, 固相体積率, ケース名 などが記載されたファイル (例: t_data.csv)
    all_features_csv: SourceCSV, MeanFeatureValue が記載されたファイル (例: all_features.csv)
    output_csv:   出力先のファイルパス

    処理の流れ:
      1) t_data.csv を読み込み
         - 列を ["計測日","固相体積率","ケース名"] などにリネーム (必要に応じて)
      2) all_features.csv を読み込み
         - SourceCSV から "_tdx数字.csv" を除去し, '_' を '-' に置き換えてケース名を抽出
      3) t_data の順序を保つように merge
         - マッチしなかった行(ケース名不一致)は削除(=放置)
      4) CSV 出力
    """

    # --- 1) t_data.csv を読み込み ---
    # もし実際に「0,5,2」みたいな列インデックスになっているなら、列名を設定しておく
    # header=None なら最初の行がデータとして扱われる。最初の行に列名が入っているなら header=0 にする。
    # 下記は例として、最初の行を列名に使う（header=0）ケース:
    t_df = pd.read_csv(t_data_csv, header=0)  
    # あるいは状況に応じて ↓ のように書き換える:
    # t_df = pd.read_csv(t_data_csv, header=None, names=["計測日","固相体積率","ケース名"])

    # 列名を念のため揃えておく（例: インデックスが "0","5","2" となっているなら書き換え）
    # 実際の列順に合わせて修正してください:
    # 例: t_df.columns = ["計測日", "固相体積率[-]", "ケース名"]
    # ここでは既に正しく列名が割り当てられていると想定
    
    print("t_data columns:", t_df.columns)

    # t_data 側に「並び順を保持するための」列を追加
    t_df["__order"] = range(len(t_df))

    # --- 2) all_features.csv を読み込み ---
    f_df = pd.read_csv(all_features_csv)
    # 例: f_df.columns => ["SourceCSV","MeanFeatureValue"]

    # 「SourceCSV」列からケース名を抽出
    # ここで "_tdx数字.csv" を削除し、 '_' を '-' に置き換える
    def extract_case_name(src):
        # 例: "s1_g0_l1_t1_tdx1.csv" -> "s1_g0_l1_t1"
        # "_tdx任意の数字.csv" を除去
        # さらに '_' を '-' に置き換える: "s1_g0_l1_t1" -> "s1-g0-l1-t1"
        name = re.sub(r'_tdx\d+\.csv$', '', src)  # 末尾の "_tdx...csv" を削除
        name = name.replace('_','-')              # アンダーバーをハイフンに
        return name
    
    f_df["ケース名"] = f_df["SourceCSV"].apply(extract_case_name)

    print("all_features columns:", f_df.columns)

    # --- 3) merge ---
    # t_data を左にして merge し、共通しないケース名は落としたい => 内部結合 "inner"
    # ただし t_data の順序を保ちたい => 後でソートし直す
    merged = pd.merge(
        t_df, f_df,
        on="ケース名",
        how="inner"  # マッチしない行は削除
    )

    # t_data の元の並び順でソートし、補助列__orderを削除
    merged = merged.sort_values("__order").drop(columns="__order")

    # --- 4) CSV 出力 ---
    merged.to_csv(output_csv, index=False, encoding="utf-8")
    print(f"Done. Merged CSV saved to: {output_csv}")


if __name__ == "__main__":
    t_data_csv     = "//mnt/sdb/matsubara/target/t_data.csv"
    all_features   = "//mnt/sdb/matsubara/input/all_features_standardized.csv"
    output_csv     = "//mnt/sdb/matsubara/input/matched.csv"

    match_and_order(t_data_csv, all_features, output_csv)


t_data columns: Index(['0', '5', '2'], dtype='object')
all_features columns: Index(['SourceCSV', 'MeanFeatureValue', 'ケース名'], dtype='object')


KeyError: 'ケース名'